In [1]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import numpy
from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array, load_img
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,BatchNormalization
from keras.applications.vgg19 import VGG19, preprocess_input
import keras.backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras import Input, Model
import PIL
from PIL import Image,ImageOps
from PIL import ImageDraw 
import shutil
import os
from os import listdir
from PIL import Image
import xml.etree.ElementTree as ET
from tqdm import tqdm
import seaborn as sns
import string
import nltk
from nltk.tokenize import word_tokenize
import pickle
import re
from numpy import savez_compressed

ModuleNotFoundError: No module named 'cv2'

Tag Prediction

In [2]:
train=pd.read_csv('traindata.csv',nrows=20)
test=pd.read_csv('testdata.csv',nrows=10)

In [3]:
train.head()

,Unnamed: 0,patient_id,image1_id,image2_id,report
0,0,CXR3080_IM-1440,CXR3080_IM-1440-1001,CXR3080_IM-1440-1002,startseq the cavity and the left upper lobe ha...
1,1,CXR2721_IM-1183,CXR2721_IM-1183-1001,CXR2721_IM-1183-2001,startseq heart size at the upper limits of nor...
2,2,CXR2160_IM-0778,CXR2160_IM-0778-1001,CXR2160_IM-0778-2001,startseq the cardiomediastinal silhouette is w...
3,3,CXR3586_IM-1764,CXR3586_IM-1764-1001,CXR3586_IM-1764-2001,startseq the heart is mildly enlarged the medi...
4,4,CXR3139_IM-1476,CXR3139_IM-1476-1001,CXR3139_IM-1476-2001,startseq no focal areas of consolidation no pl...


In [4]:
train.drop('Unnamed: 0',axis='columns',inplace=True)
test.drop('Unnamed: 0',axis='columns',inplace=True)

In [5]:
train.head()

,patient_id,image1_id,image2_id,report
0,CXR3080_IM-1440,CXR3080_IM-1440-1001,CXR3080_IM-1440-1002,startseq the cavity and the left upper lobe ha...
1,CXR2721_IM-1183,CXR2721_IM-1183-1001,CXR2721_IM-1183-2001,startseq heart size at the upper limits of nor...
2,CXR2160_IM-0778,CXR2160_IM-0778-1001,CXR2160_IM-0778-2001,startseq the cardiomediastinal silhouette is w...
3,CXR3586_IM-1764,CXR3586_IM-1764-1001,CXR3586_IM-1764-2001,startseq the heart is mildly enlarged the medi...
4,CXR3139_IM-1476,CXR3139_IM-1476-1001,CXR3139_IM-1476-2001,startseq no focal areas of consolidation no pl...


In [6]:
test.head()

,patient_id,image1_id,image2_id,report
0,CXR1306_IM-0200,CXR1306_IM-0200-2001,CXR1306_IM-0200-3001,startseq the examination consists of frontal ...
1,CXR1786_IM-0512,CXR1786_IM-0512-1001,CXR1786_IM-0512-2001,startseq the lungs are clear bilaterally speci...
2,CXR2072_IM-0706,CXR2072_IM-0706-1001,CXR2072_IM-0706-2001,startseq stable nonenlarged cardiomediastinal ...
3,CXR467_IM-2095,CXR467_IM-2095-2001,CXR467_IM-2095-2001,startseq lungs are clear bilaterally cardiac a...
4,CXR3087_IM-1444,CXR3087_IM-1444-1001,CXR3087_IM-1444-2001,startseq the heart is enlarged the mediastinal...


In [255]:
with open('train_features_list_new.pkl', 'rb') as f:
    train_feat_list = pickle.load(f)

In [256]:
with open('test_features_list_new.pkl', 'rb') as f:
    test_feat_list = pickle.load(f)

In [257]:
train_feat_list_new = []
for i in range(0,20):
    train_feat_list_new.append(train_feat_list[i])

In [258]:
test_feat_list_new = []
for i in range(0,10):
    test_feat_list_new.append(test_feat_list[i])

In [259]:
len(train_feat_list_new), len(test_feat_list_new)

(20, 10)

In [15]:
df_tags = pd.read_csv("tags.csv")

In [16]:
df_tags.head()

,tags,patient_id
0,normal,CXR3698_IM-1846
1,normal,CXR370_IM-1848
2,normal,CXR3679_IM-1831
3,Stents coronary vessels,CXR3689_IM-1840
4,normal,CXR3664_IM-1822


In [17]:
df_tags.reset_index(drop=True, inplace=True)

In [18]:
df_new_2 = pd.DataFrame(train.merge(df_tags, on='patient_id', how='left'))

In [19]:
df_new_2.describe()

,patient_id,image1_id,image2_id,report,tags
count,20,20,20,20,20
unique,20,20,20,20,13
top,CXR2721_IM-1183,CXR2407_IM-0954-1001,CXR2721_IM-1183-2001,startseq the cardiomediastinal silhouette and ...,normal
freq,1,1,1,1,8


In [20]:
df_new_2.head()

,patient_id,image1_id,image2_id,report,tags
0,CXR3080_IM-1440,CXR3080_IM-1440-1001,CXR3080_IM-1440-1002,startseq the cavity and the left upper lobe ha...,Cavitation lung upper lobe right
1,CXR2721_IM-1183,CXR2721_IM-1183-1001,CXR2721_IM-1183-2001,startseq heart size at the upper limits of nor...,Deformity thoracic vertebrae multiple mild
2,CXR2160_IM-0778,CXR2160_IM-0778-1001,CXR2160_IM-0778-2001,startseq the cardiomediastinal silhouette is w...,normal
3,CXR3586_IM-1764,CXR3586_IM-1764-1001,CXR3586_IM-1764-2001,startseq the heart is mildly enlarged the medi...,Cardiomegaly mild
4,CXR3139_IM-1476,CXR3139_IM-1476-1001,CXR3139_IM-1476-2001,startseq no focal areas of consolidation no pl...,normal


In [21]:
df_new_2.isnull().sum()

patient_id    0
image1_id     0
image2_id     0
report        0
tags          0
dtype: int64

In [22]:
df_new_2.head()

,patient_id,image1_id,image2_id,report,tags
0,CXR3080_IM-1440,CXR3080_IM-1440-1001,CXR3080_IM-1440-1002,startseq the cavity and the left upper lobe ha...,Cavitation lung upper lobe right
1,CXR2721_IM-1183,CXR2721_IM-1183-1001,CXR2721_IM-1183-2001,startseq heart size at the upper limits of nor...,Deformity thoracic vertebrae multiple mild
2,CXR2160_IM-0778,CXR2160_IM-0778-1001,CXR2160_IM-0778-2001,startseq the cardiomediastinal silhouette is w...,normal
3,CXR3586_IM-1764,CXR3586_IM-1764-1001,CXR3586_IM-1764-2001,startseq the heart is mildly enlarged the medi...,Cardiomegaly mild
4,CXR3139_IM-1476,CXR3139_IM-1476-1001,CXR3139_IM-1476-2001,startseq no focal areas of consolidation no pl...,normal


In [23]:
df_new_2['tags'].replace('\d+', '', regex=True, inplace=True)

In [24]:
df_new_2.head()

,patient_id,image1_id,image2_id,report,tags
0,CXR3080_IM-1440,CXR3080_IM-1440-1001,CXR3080_IM-1440-1002,startseq the cavity and the left upper lobe ha...,Cavitation lung upper lobe right
1,CXR2721_IM-1183,CXR2721_IM-1183-1001,CXR2721_IM-1183-2001,startseq heart size at the upper limits of nor...,Deformity thoracic vertebrae multiple mild
2,CXR2160_IM-0778,CXR2160_IM-0778-1001,CXR2160_IM-0778-2001,startseq the cardiomediastinal silhouette is w...,normal
3,CXR3586_IM-1764,CXR3586_IM-1764-1001,CXR3586_IM-1764-2001,startseq the heart is mildly enlarged the medi...,Cardiomegaly mild
4,CXR3139_IM-1476,CXR3139_IM-1476-1001,CXR3139_IM-1476-2001,startseq no focal areas of consolidation no pl...,normal


In [74]:
df_new_2.to_csv('df_coatt.csv')

In [329]:
def create_tag_mapping(mapping_csv):
    labels = set()
    for i in range(len(mapping_csv)):
        tags = mapping_csv['tags'][i].split(' ')
        labels.update(tags)
    labels = list(labels)
    labels.sort()
    labels_map = {labels[i]:i for i in range(len(labels))}
    inv_labels_map = {i:labels[i] for i in range(len(labels))}
    return labels_map, inv_labels_map
 
def create_file_mapping(mapping_csv):
    mapping = dict()
    for i in range(len(mapping_csv)):
        name, tags = mapping_csv['image1_id'][i], mapping_csv['tags'][i]
        mapping[name] = tags.split(' ')
    return mapping
 

def one_hot_encode(tags, mapping):
    encoding = np.zeros(len(mapping), dtype='uint8')
    for tag in tags:
        encoding[mapping[tag]] = 1
    return encoding
 
def load_dataset(path, file_mapping, tag_mapping):
    photos, targets = list(), list()
    for i in file_mapping:
        for filename in os.listdir(folder):
            image_name = os.path.splitext(filename)[0]
            if(i==image_name):
                photo = Image.open(path + image_name + '.png')
                photo=img_to_array(photo)
                photo = preprocess_input(photo)
                photo=cv2.resize(photo,(128,128))
                photo = img_to_array(photo, dtype='uint8')
                tags = file_mapping[image_name]
                target = one_hot_encode(tags, tag_mapping)
                photos.append(photo)
                targets.append(target)
    X = np.asarray(photos, dtype='uint8')
    y = np.asarray(targets, dtype='uint8')
    return X, y
 
filename = 'df_coatt.csv'
mapping_csv = pd.read_csv(filename)
tag_mapping, _ = create_tag_mapping(mapping_csv)
file_mapping = create_file_mapping(mapping_csv)
folder = 'C:/Users/INDIRA/Downloads/data rescon/images/'
X, y = load_dataset(folder, file_mapping, tag_mapping)
print(X.shape, y.shape)
savez_compressed('data_coatt.npz', X, y)

(20, 128, 128, 3) (20, 30)


In [216]:
def load_dataset():
    data = np.load('data_coatt.npz')
    X, y = data['arr_0'], data['arr_1']
    trainX, trainY = X, y
    return trainX, trainY

In [217]:
trainX.shape , trainY.shape

((20, 128, 128, 3), (20, 30))

In [212]:
def define_model(in_shape=(128, 128, 3), out_shape=30):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(out_shape, activation='softmax'))
    # compile model
    opt = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics='accuracy')
    return model

In [218]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
trainX, trainY = load_dataset()
train_it = datagen.flow(trainX, trainY, batch_size=5)
model = define_model()
history = model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=10, verbose=0)

In [219]:
loss, fbeta = model.evaluate_generator(train_it, steps=len(train_it), verbose=0)
print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))

> loss=0.225, fbeta=0.500


In [220]:
yhat = model.predict(trainX)

In [221]:
yhat.shape

(20, 30)

In [222]:
yhat_list = []

In [223]:
yhat_list = yhat.max(axis=1)

In [224]:
len(yhat_list)

20

In [225]:
yhat_list

array([1.        , 0.89129025, 1.        , 1.        , 1.        ,
       1.        , 0.9939778 , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.99420446, 0.99827516, 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ],
      dtype=float32)

In [226]:
labels = np.argmax(yhat, axis=-1) 

In [227]:
labels

array([24, 22, 24, 22, 24, 22,  8, 24, 24, 24, 24, 24,  6, 21, 22, 24, 24,
       24, 21, 24], dtype=int64)

In [230]:
_, inv_tag_mapping = create_tag_mapping(mapping_csv)

In [231]:
inv_tag_mapping

{0: 'Airspace',
 1: 'Bone',
 2: 'Bones',
 3: 'Cardiomegaly',
 4: 'Cavitation',
 5: 'Cicatrix',
 6: 'Deformity',
 7: 'Density',
 8: 'Disease',
 9: 'Granulomatous',
 10: 'Hernia,',
 11: 'Hiatal',
 12: 'Lung',
 13: 'Thoracic',
 14: 'Vertebrae',
 15: 'and',
 16: 'base',
 17: 'degenerative',
 18: 'hypoinflation',
 19: 'left',
 20: 'lobe',
 21: 'lung',
 22: 'mild',
 23: 'multiple',
 24: 'normal',
 25: 'right',
 26: 'thoracic',
 27: 'thorax',
 28: 'upper',
 29: 'vertebrae'}

In [331]:
semantic_feat = []

In [332]:
for keys in inv_tag_mapping.keys():
    for item in labels:
        if item == keys:
            semantic_feat.append(inv_tag_mapping[keys])

In [333]:
semantic_feat

['Deformity',
 'Disease',
 'lung',
 'lung',
 'mild',
 'mild',
 'mild',
 'mild',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal']

In [335]:
 tag_mapping, _ = create_tag_mapping(mapping_csv)

In [337]:
semantic_encoding = one_hot_encode(semantic_feat,tag_mapping)

In [338]:
semantic_feat_encoding = list(semantic_encoding)

Semantic Features

In [339]:
len(semantic_feat_encoding)

30

In [340]:
co_att_feat = []

Visual Features

In [341]:
len(train_feat_list_new)

20

Concatenating two features

In [342]:
co_att_feat = train_feat_list_new + semantic_feat_encoding

In [343]:
len(co_att_feat)

50

In [344]:
co_att_feat_arr = x = np.asarray(co_att_feat)

In [345]:
co_att_feat_arr.shape

(50,)

In [350]:
type(co_att_feat_arr)

numpy.ndarray

Co-attention layer to generate context vector

In [348]:
inp = Input(shape=co_att_feat_arr.shape[0])
hidden = Dense(8, activation='relu')(inp)
flat = Flatten()(hidden)
out = Dense(2)(flat)
model = Model(inputs=inp, outputs=out)